In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W49`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W50`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W51`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W52`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W53`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W01`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W02`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W03`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W04`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W05`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W06`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W07`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.sort_values(by=['Week']).tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
4922,2021-02-12 09:36:10+00:00,2021-02-12 09:43:08+00:00,Production,None,L4,None,None,KL13,None,Friday Morning,iXUsr_Anne(7488),3,6.97,2021-07
4923,2021-02-14 07:55:40+00:00,2021-02-14 08:00:36+00:00,Preparation/Changeover,None,S2,None,None,KL18,None,Sunday Morning,iXUsr_Kim Jie Yap 7055;,3,4.93,2021-07
41784,2021-02-14 09:39:37+00:00,2021-02-14 09:43:35+00:00,Production,0019 of 0074,C2,None,None,KL3,Tue-NI Delivery,Sunday Morning,iXUsr_Kim Jie Yap 7055;,3,3.97,2021-07
33924,2021-02-12 18:45:34+00:00,2021-02-12 18:48:21+00:00,Production,0008 of 0011,V2,None,None,KL15,Sun-NI Delivery,Friday Afternoon,Melanie Dua;,4,2.78,2021-07
31384,2021-02-11 15:58:42+00:00,2021-02-11 16:01:55+00:00,Production,0027 of 0028,L4,None,None,KL13,Sat-NI Delivery,Thursday Afternoon,iXUsr_Peni(8294),3,3.22,2021-07


In [5]:
data_in_use.shape

(62770, 14)